In [1]:
import os
from glob import glob
import numpy as np
import pandas as pd 
import shutil
import sys
import geopandas as gpd

#home = "/share/klmarti3/kmcquil/Chapter2_mechanisms_forest_water_cycling"
home = "/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling"
exec(open(os.path.join(home, 'Scripts', "Functions.py")).read())

In [2]:
trends = pd.read_csv(os.path.join(home, 'Data', 'NDVI', 'ratio_ndvi_trend_results.csv'))
ratio_trends = trends[trends['metric'] == 'trend_ratio_ndvi']
sd_trends = trends[trends['metric'] == 'trend_sd_ndvi']

print(trends.shape)
print(ratio_trends.shape)
print(sd_trends.shape)

(60088, 11)
(30044, 11)
(30044, 11)


In [7]:
np.mean(ratio_trends.years)
2020-1984

36

In [36]:
sig_ratio = ratio_trends[(ratio_trends['p_value'] <= 0.05)]
np.quantile(sig_ratio.slope, 0.75)

-0.0005117832204249251

In [4]:
sig_neg_ratio = ratio_trends[(ratio_trends['p_value'] < 0.05) & (ratio_trends['slope'] < 0)]
sig_pos_ratio = ratio_trends[(ratio_trends['p_value'] < 0.05) & (ratio_trends['slope'] > 0)]
non_sig_ratio = ratio_trends[(ratio_trends['p_value'] >= 0.05)]


print(sig_neg_ratio.shape[0] / ratio_trends.shape[0])
print(sig_pos_ratio.shape[0] / ratio_trends.shape[0])
print(non_sig_ratio.shape[0] / ratio_trends.shape[0])

0.44301690853414993
0.08334442817201437
0.4736386632938357


In [5]:
sig_neg_sd = sd_trends[(sd_trends['p_value'] < 0.05) & (sd_trends['slope'] < 0)]
sig_pos_sd = sd_trends[(sd_trends['p_value'] < 0.05) & (sd_trends['slope'] > 0)]
non_sig_sd = sd_trends[(sd_trends['p_value'] >= 0.05)]



print(sig_neg_sd.shape[0] / sd_trends.shape[0])
print(sig_pos_sd.shape[0] / sd_trends.shape[0])
print(non_sig_sd.shape[0] / sd_trends.shape[0])

0.2751630941286114
0.09719078684595926
0.6276461190254293


In [6]:
print(np.mean(trends['n']))
print(np.mean(trends['years']))

69.68176674211156
29.87138863000932


In [7]:
# create a column of masked slopes by p value for ratio ndvi and sd ndvi 
ratio_trends['rslopesig'] = ratio_trends['slope']
ratio_trends.rslopesig = np.where(ratio_trends.p_value <= 0.05, ratio_trends.rslopesig, np.nan)

sd_trends['sdslopesig'] = sd_trends['slope']
sd_trends.sdslopesig = np.where(sd_trends.p_value <= 0.05, sd_trends.sdslopesig, np.nan)

/var/folders/6_/17bx0qz13sjdf13yymjj7kb00000gn/T/ipykernel_22073/775265132.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratio_trends['rslopesig'] = ratio_trends['slope']
/var/folders/6_/17bx0qz13sjdf13yymjj7kb00000gn/T/ipykernel_22073/775265132.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratio_trends.rslopesig = np.where(ratio_trends.p_value <= 0.05, ratio_trends.rslopesig, np.nan)
/var/folders/6_/17bx0qz13sjdf13yymjj7kb00000gn/T/ipykernel_22073/775265132.py:5: SettingWithCopyWarning: 
A valu

In [19]:
# merge the masked significant value with the 100% forested headwater shapefile 
hws = gpd.read_file(os.path.join(home, "Data", "Catchments", "Headwater", "headwater_catchments_perm_forest.shp"))

hws_updated = hws.merge(ratio_trends[['wsid', 'rslopesig']], left_on="NHDPlusID", right_on = "wsid",how = 'left')
hws_updated = hws_updated.drop('wsid', 1)
hws_updated = hws_updated.dropna(subset='rslopesig')
hws_updated.to_file(os.path.join(home, "Data", "Catchments", "Headwater", "headwater_catchments_sig_trend.shp"))

/var/folders/6_/17bx0qz13sjdf13yymjj7kb00000gn/T/ipykernel_22073/626516038.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  hws_updated = hws_updated.drop('wsid', 1)
/Users/katiemcquillan/opt/anaconda3/envs/ch2-python-env/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [21]:
#print(np.nanmin(hws_updated.rslopesig))
#print(np.nanmax(hws_updated.rslopesig))


print(np.nanquantile(hws_updated.rslopesig, 0))
print(np.nanquantile(hws_updated.rslopesig, 0.25))
print(np.nanquantile(hws_updated.rslopesig, 0.50))
print(np.nanquantile(hws_updated.rslopesig, 0.75))
print(np.nanquantile(hws_updated.rslopesig, 0.99))
print(np.nanquantile(hws_updated.rslopesig, 1))



-0.0059675995685937
-0.001227181823866125
-0.00083040733500145
-0.0005117832204249251
0.0018778083183537701
0.0076022406022649


### Calculate the % area and area weighted mean of NDVI trends in each reference watershed 

Find the intersecting area 

Calculate the total intersecting area 

Divide by the total area of the reference watershed 

In [9]:
home = "/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling"
#hws = gpd.read_file(os.path.join(home, "Data", "Catchments", "Headwater", "headwater_catchments_sig_trend.shp"))
hws = gpd.read_file(os.path.join(home, "Data", "Catchments", "Headwater", "headwater_catchments_perm_forest_32617.shp"))
hws = hws.to_crs("epsg:32617")
sbr_ref_combo_diss = gpd.read_file(os.path.join(home, "Data", "Catchments", "Reference", "gages_ii","reference_keep.shp"))
refs = sbr_ref_combo_diss.to_crs(hws.crs)
refs['AREA'] = refs['AREA'].multiply(1e-6) # convert area from m2 to km2 
gages = refs['GAGE_ID']

In [12]:
rows = []
for i in gages:
    ref_poly = refs[refs['GAGE_ID'] == i]
    results = gpd.overlay(ref_poly, hws, how='intersection')
    results['updated_area'] = results.apply(lambda row: row.geometry.area,axis=1)
    results['updated_area'] = results['updated_area'].multiply(1e-6)

    pct_area = np.sum(results['updated_area']) / results.AREA.iloc[0]
    total_area = np.sum(results['updated_area'])
    #weighted_avg = np.sum(results['rslopesig'] * results['updated_area'].divide(total_area))
    dic = {'Gage':i, 'pct_hw': pct_area}
    rows.append(dic)
ref_results = pd.DataFrame(rows)

In [19]:
hw_pct_dt = refs.merge(ref_results, left_on='GAGE_ID', right_on='Gage')
hw_pct_dt_final = hw_pct_dt[hw_pct_dt['pct_stream'] == 100]
np.mean(hw_pct_dt_final.pct_hw)

0.08610877838408448

In [20]:
hw_pct_dt_final


,AREA,PERIMETER,GAGE_ID,index_righ,NA_L3NAME,pct_stream,geometry,Gage,pct_hw
0,280.7200,153660.0,02053800,0.0,Blue Ridge,100.0,"POLYGON ((579510.716 4113911.004, 579505.951 4...",02053800,0.046646
1,298.1070,135960.0,02056900,0.0,Blue Ridge,100.0,"POLYGON ((579439.826 4111681.306, 579444.591 4...",02056900,0.005038
2,221.1330,110700.0,02069700,0.0,Blue Ridge,100.0,"POLYGON ((570855.123 4056381.301, 570850.357 4...",02069700,0.037141
3,270.6080,117480.0,02070000,0.0,Blue Ridge,100.0,"POLYGON ((582963.340 4065335.883, 582993.239 4...",02070000,0.010336
5,233.6610,115980.0,02111500,0.0,Blue Ridge,100.0,"POLYGON ((469487.263 4025044.732, 469492.017 4...",02111500,0.012388
8,400.5270,162660.0,02118500,0.0,Blue Ridge,100.0,"POLYGON ((511444.728 4002154.727, 511449.487 4...",02118500,0.002230
9,329.5440,110520.0,02137727,0.0,Blue Ridge,100.0,"POLYGON ((396155.936 3952163.345, 396160.676 3...",02137727,0.268927
12,216.6720,130500.0,02143000,0.0,Blue Ridge,100.0,"POLYGON ((457776.895 3951596.739, 457781.644 3...",02143000,0.023476
13,66.4841,37832.1,02143040,0.0,Blue Ridge,100.0,"POLYGON ((443375.925 3935972.369, 443328.994 3...",02143040,0.086979
14,203.8530,101820.0,02149000,0.0,Blue Ridge,100.0,"POLYGON ((407956.387 3937665.652, 407926.500 3...",02149000,0.011451


In [79]:
# find the area of the permanently forested headwaters within a given reference watershed 
rows = []
for i in gages:
    ref_poly = refs[refs['GAGE_ID'] == i]
    results = gpd.overlay(ref_poly, hws, how='intersection')
    results['updated_area'] = results.apply(lambda row: row.geometry.area,axis=1)
    results['updated_area'] = results['updated_area'].multiply(1e-6)

    pct_area = np.sum(results['updated_area']) / results.AREA.iloc[0]
    total_area = np.sum(results['updated_area'])
    weighted_avg = np.sum(results['rslopesig'] * results['updated_area'].divide(total_area))
    dic = {'Gage':i, 'pct_hw': pct_area, 'wavg_rslopesig': weighted_avg}
    rows.append(dic)
ref_results = pd.DataFrame(rows)
ref_results.to_csv(os.path.join(home, "Data", "NDVI", "reference_watershed_ndvi_trend_summary.csv"), index=False)

#tryy = hws[hws['NHDPlusID'].isin(results['NHDPlusID'])]
#tryy.explore()
#results.explore()

In [32]:
## get the overall ratio 
ndvi_summary_long = pd.read_csv(os.path.join(home, "Data", "NDVI", "annual_ratio_ndvi.csv" ))
#ndvi_summary_long = ndvi_summary_long.groupby('WSID').mean().reset_index()
#ndvi_summary_long
#np.median(ndvi_summary_long.RatioDownUp_NDVI)
#np.quantile(ndvi_summary_long.RatioDownUp_NDVI, 0.25)
#np.quantile(ndvi_summary_long.RatioDownUp_NDVI, 0.75)

ndvi_summary_long


,year,RatioDownUp_NDVI,Mean_NDVI,WSID
0,1989,0.987909,0.467099,15000100000037
1,2011,1.011625,0.451514,15000100000037
2,2005,0.981766,0.497167,15000100000037
3,2018,0.963539,0.494422,15000100000037
4,2003,0.962360,0.439196,15000100000037
...,...,...,...,...
909808,1996,0.916003,0.469375,25000400269281
909809,1989,0.929455,0.429897,25000400269281
909810,1994,0.930070,0.409385,25000400269281
909811,1999,0.905004,0.422873,25000400269281


In [41]:
trends = pd.read_csv(os.path.join(home, 'Data', 'NDVI', 'ratio_ndvi_trend_results_outliers.csv'))
ratio_trends = trends[trends['metric'] == 'trend_ratio_ndvi']
sd_trends = trends[trends['metric'] == 'trend_sd_ndvi']
downslope_trends = trends[trends['metric'] == 'trend_downslope_ndvi']
upslope_trends = trends[trends['metric'] == 'trend_upslope_ndvi']


print(trends.shape)
print(ratio_trends.shape)
print(sd_trends.shape)
print(downslope_trends.shape)
print(upslope_trends.shape)

(120180, 11)
(30045, 11)
(30045, 11)
(30045, 11)
(30045, 11)


In [42]:
sig_neg_ratio = ratio_trends[(ratio_trends['p_value'] < 0.05) & (ratio_trends['slope'] < 0)]
sig_pos_ratio = ratio_trends[(ratio_trends['p_value'] < 0.05) & (ratio_trends['slope'] > 0)]
non_sig_ratio = ratio_trends[(ratio_trends['p_value'] >= 0.05)]


print(sig_neg_ratio.shape[0] / ratio_trends.shape[0])
print(sig_pos_ratio.shape[0] / ratio_trends.shape[0])
print(non_sig_ratio.shape[0] / ratio_trends.shape[0])

0.48457314028956566
0.09592278249292728
0.4195040772175071


In [43]:
sig_neg_sd = sd_trends[(sd_trends['p_value'] < 0.05) & (sd_trends['slope'] < 0)]
sig_pos_sd = sd_trends[(sd_trends['p_value'] < 0.05) & (sd_trends['slope'] > 0)]
non_sig_sd = sd_trends[(sd_trends['p_value'] >= 0.05)]


print(sig_neg_sd.shape[0] / sd_trends.shape[0])
print(sig_pos_sd.shape[0] / sd_trends.shape[0])
print(non_sig_sd.shape[0] / sd_trends.shape[0])

0.3620901980362789
0.11506074222000333
0.5228490597437178


In [44]:
sig_neg_downslope = downslope_trends[(downslope_trends['p_value'] < 0.05) & (downslope_trends['slope'] < 0)]
sig_pos_downslope = downslope_trends[(downslope_trends['p_value'] < 0.05) & (downslope_trends['slope'] > 0)]
non_sig_downslope = downslope_trends[(downslope_trends['p_value'] >= 0.05)]

print(sig_neg_downslope.shape[0] / downslope_trends.shape[0])
print(sig_pos_downslope.shape[0] / downslope_trends.shape[0])
print(non_sig_downslope.shape[0] / downslope_trends.shape[0])

0.07295723082043601
0.2200033283408221
0.7070394408387419


In [45]:
sig_neg_upslope = upslope_trends[(upslope_trends['p_value'] < 0.05) & (upslope_trends['slope'] < 0)]
sig_pos_upslope = upslope_trends[(upslope_trends['p_value'] < 0.05) & (upslope_trends['slope'] > 0)]
non_sig_upslope = upslope_trends[(upslope_trends['p_value'] >= 0.05)]

print(sig_neg_upslope.shape[0] / upslope_trends.shape[0])
print(sig_pos_upslope.shape[0] / upslope_trends.shape[0])
print(non_sig_upslope.shape[0] / upslope_trends.shape[0])

0.036711599267765016
0.3406556831419537
0.6226327175902813
